<a href="https://colab.research.google.com/github/JairEsc/Mat_Apl/blob/main/Opt_Proyecto_derivative_free.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Idea general del algoritmo.

Input: 


*   $f:\mathbb{R}^n⟶ \mathbb{R}$, función objetivo.
*   $x_0$, aproximación inicial.
*   $a_{max}$, Número máximo de iteraciones para la busqueda en linea.
*   $\zeta$, parámetro de curvatura.
*   $tol_g$, criterio de paro para la norma del gradiente.



Descripción general del algoritmo

*   Calcular $f(x_0)$
*   Estimar $\epsilon_f$ usando ECnoise*
*   Calcular ** h
*   Calcular $\nabla_h f(x_0)$. Cuando se haga, guardar los valores $(x_s,f_s)$ que satisfacen $f_s=f(x_s)=min_{x\in S} f(x)$, $S=\{x+h\cdot e_i,i=1,\ldots,n\}$
*   While ($||\nabla f(x_k)||>tol_g$):
    *   Calcular $d_k=-H_k\nabla_h f(x_k)$ usando L-BFGS***
    *   Hacer line_search$(x_k,f_k,\nabla_h f(x_k),d_k,a_{max})$ y obtiene $(x_+,f_+,\alpha_k,LS_{flag})$
        *   If($LS_{flag}==1$):
            *   Invocar Recovery($x_s,f_s$)
        *   Else:
            *   Actualizar $x_{k+1}=x_+,$ $f_{k+1}=f_+$
    *   Calcular $\nabla_h f(x_{k+1})$ y guardar $(x_s,f_s)$
    *   Calcular $s_k=x_{k+1}-x_k$, $y_k=\nabla_h f(x_{k+1})-\nabla_h f(x_k)$
        *   Guardarlos si $s_k^Ty_k\geq \zeta||s_k||||y_k||$

Explicación de L-BFGS:
Si $H_k$ es una aproximación de la inversa del Hessiano, nos interesa caluclar $d_k=-H_k\nabla f(x_k)$.


Input:



*   $\nabla f(x_k)$
*   $\{(s_{k-1},y_{k-1}),(s_{k-2},y_{k-2}),\ldots,(s_{k-m},y_{k-m})\}$
*   $H^0_{k-m}$

Algoritmo:


*   Define $q=\nabla f(x_k)$
*   for $i=1,2,\ldots,m$ do:
    *   Calcular $\alpha_i=\rho_i s_{k-i}^Tq$
    *   Actualiza $q=q-\alpha_iy_i$
*   Define $r=H^0_{k-m}q$
*   for $i=1,2,\ldots,m$ do:
    *   Calcular $\beta=\rho_{k-i}y_{k-i}^Tr$
    *   Actualiza $r=r-s_{k-i}(\alpha_i-\beta)$
*   $r$ es una aproximación de $H_k\nabla f(x_k)$


In [12]:
import numpy as np
import scipy.optimize

In [13]:
def L_BFGS(S,Y,g):
    n=len(g)
    m=len(S)
    #supondremos H_0 un multiplo de la identidad
    H_0=(np.dot(S[-1],Y[-1])/(np.dot(Y[-1],Y[-1])))*np.identity(n)#Dado en clase.
    q=g
    alphas=[]
    for i in range(m):
        alphas.append((1/np.dot(Y[-i],S[-i]))*np.dot(S[-i],q))
        q=q-alphas[i]*Y[-i]
    r=np.dot(H_0,q)
    for i in range(m):
        beta=(1/np.dot(Y[-i],S[-i]))*np.dot(Y[-i],r)
        r=r+S[-i]*(alphas[i]-beta)#aproximacion de -H*g
    return r

In [14]:
def ECNoise(F):#F es un vector que guarda f(t_i), i=0,...,m. Evaluaciones de m+1 puntos equiespaciados
    m=len(F)-1
    T=np.zeros((m+1,m+1))
    for i in range(m+1):
        T[i,0]=F[i]
    for k in range(m):
        for i in range(m-k):
            T[i,k+1]=T[i+1,k]-T[i,k]
    return T

In [15]:
def var_estimation(T):#Recibe la tabla.
    m=len(T)-1
    sigma_array=[]
    for k in range(1,m):
        sigma_array.append(((np.math.factorial(k)/np.math.factorial(2*k))/(m+1-k))*np.sum(T.T[k]**2))
    #Ahora checamos las 2 condiciones.
    for k in range(1,m-2):
        max_k=np.max(sigma_array[k:k+2])
        min_k=np.min(sigma_array[k:k+2])
        if(max_k<=4*min_k):
            if(np.sign(min_k*max_k)==-1):
                print("es el ", k)
                break
    return np.sqrt(sigma_array[k])#Podria ser que las condiciones no se cumplan, en tal caso regresa la ultima estimacion sigma.

In [47]:
def aprox_f_biprima(e_f,f,x_k,p):#Usando una direccion (aleatoria dada) se aproxima el max ||Hessiana_f(x_k)*p||
    tao_1=100
    tao_2=0.1
    h_a=e_f**(1/4)
    f_mas=f(x_k+h_a*p)
    f_menos=f(x_k-h_a*p)
    f_0=f(x_k)
    delta_h_a=abs(f_mas+f_menos-2*f_0)
    mu_a=delta_h_a/(h_a**2)
    if(delta_h_a/e_f>=tao_1 and abs(f_mas-f_0)<=tao_2*max(f_mas,f_menos,f_0) and abs(f_menos-f_0)<=tao_2*max(f_mas,f_menos,f_0)):
        return mu_a
    h_b=(e_f/mu_a)**(1/4)
    f_mas=f(x_k+h_b*p)
    f_menos=f(x_k-h_b*p)
    delta_h_b=abs(f_mas+f_menos-2*f_0)
    mu_b=delta_h_b/(h_b**2)
    if(delta_h_b/e_f>=tao_1 and abs(f_mas-f_0)<=tao_2*max(f_mas,f_menos,f_0) and abs(f_menos-f_0)<=tao_2*max(f_mas,f_menos,f_0)):
        return mu_b
    if(abs(mu_a-mu_b)<=0.5*mu_b):
        return mu_b
    return mu_a


In [46]:
max( 0,1,2)

2

In [17]:
def grad_f(f,x_k,h):#Regresa el gradiente, y los valores minimos (x_s,f_s)
    n=len(x_k)
    g=np.zeros(n)
    f_k=f(x_k)
    for i in range(n):
        p=np.eye(1,n,i)
        if(i==0):
            f_s_1=f(x_k+h*p)
            x_s=x_k+h*p
        f_s_2=f(x_k+h*p)
        g[i]=(f_s_2-f_k)/h
        if(f_s_2<f_s_1):
            f_s_1=f_s_2
            x_s=x_k+h*p
    print 
    return (g,[x_s,f_s_1])

In [18]:
#Usare sccipy.optimize.line_search() para la busqueda en linea.

In [34]:
def Rosembrok(X):#X es un vector.
    return (np.sum(100*(X[1:]-X[:-1]**2)**2+(1-X[:-1])**2)+10**(-3)*np.random.uniform(0,1))
def Rosembrok_grad(X):
    n=len(X)
    grad=np.zeros(n)
    grad[1:-1]=-400*X[1:-1]*(X[2:]-X[1:-1]**2)+200*(X[1:-1]-X[:-2]**2)-2*(1-X[1:-1])
    grad[0]=-400*X[0]*(X[1]-X[0]**2)-2*(1-X[0])
    grad[-1]=200*(X[-1]-X[-2]**2)
    return grad

In [48]:
def find_minimum(f,x_0,m,a_max,zeta,tol_g,max_iter=10**2):
    f_0=f(x_0)
    n=len(x_0)
    p=np.eye(1,n,np.random.randint(0,n))
    delta=10**(-3)#Suggested by ECnoise algorithm.
    F=[f(x_0+delta*i*p) for i in range(m)]
    T=ECNoise(F)
    e_f=var_estimation(T)
    print("e_f: ",e_f)#Breakpoint
    mu=aprox_f_biprima(e_f=e_f,f=Rosembrok,x_k=x_0,p=p)
    h=(8**(1/4))*(e_f/mu)**(1/2)
    print("h: ", h)
    X_s=[]
    F_s=[]
    g_0=grad_f(f=Rosembrok,x_k=x_0,h=h)


In [49]:
np.random.seed(0)
x_0=np.random.uniform(0,1,size=100)
find_minimum(f=Rosembrok,x_0=x_0,m=6,a_max=100,zeta=0.5,tol_g=10**(-3))

e_f:  9.526521142657172e-05
h:  2.5331505120121852e-05


In [19]:
#Test-------------------
def f(t):
    return (np.cos(t)+np.sin(t)+10**(-3)*np.random.uniform(0,2*np.sqrt(3)))
f_vec=np.vectorize(f)
h=10**(-2)
F=f_vec([i*h for i in range(7)])
Tabla=(ECNoise(F))
for line in Tabla:
    print(' '.join(map(str, line)))
print(var_estimation(Tabla))
e_f=var_estimation()
mu=aprox_f_biprima()
h=(8**(1/4))*(e_f/mu)**(1/2)
#Ya podemos calcular la defivada direccional.
#---------------------------

1.0017026930804827 0.01007613994625145 0.0002145821670920256 -5.192146016064214e-05 -0.0009142277486757777 0.0012037215851685712 0.003212106199910636
1.011778833026734 0.010290722113343476 0.00016266070693138346 -0.0009661492088364199 0.0002894938364927935 0.004415827785079207 0.0
1.0220695551400776 0.010453382820274859 -0.0008034885019050364 -0.0006766553723436264 0.004705321621572001 0.0 0.0
1.0325229379603524 0.009649894318369823 -0.0014801438742486628 0.004028666249228374 0.0 0.0 0.0
1.0421728322787223 0.00816975044412116 0.0025485223749797115 0.0 0.0 0.0 0.0
1.0503425827228434 0.010718272819100871 0.0 0.0 0.0 0.0 0.0
1.0610608555419443 0.0 0.0 0.0 0.0 0.0 0.0
6.764119656150118e-05


TypeError: ignored